### 解压可迭代对象赋值给多个变量
利用*星号表达式拆解列表

In [2]:
#排除首尾
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

In [6]:
#星号解压语法在字符串操作的时候也会很有用，比如字符串的分割。
line = 'nobody:*:-2:-2:Unprivileged User:/var/empty:/usr/bin/false'
uname, *fields, homedir, sh = line.split(':')
uname,homedir,sh

('nobody', '/var/empty', '/usr/bin/false')

### 保留最后 N 个元素
使用 deque(maxlen=N) 构造函数会新建一个固定大小的队列。当新的元素加入并且这个队列已满的时候， 最老的元素会自动被移除掉。

In [11]:
from collections import deque
q = deque(maxlen=3)
q.append(1)
q.append(1)
q.append(1)
print(q)
q.append(4)
print(q)

deque([1, 1, 1], maxlen=3)
deque([1, 1, 4], maxlen=3)
